# GFP RFP
Read in data, discard NaN values, then calculate slopes of different neurons, residuals, and deconvolve to get neural activity

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import interpn
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.io as sio
%pylab inline

In [ ]:
# load data
#directory = 'Data/GFP_moving/' # for Katherine's computer
directory = '../Data/20160107_AllData/GFP_moving/' # for Nick's computer
file_name = 'heatData'
heatData = sio.loadmat(directory+file_name) # Contains: DmatAll, G2, R2, Ratio2, acorr, cgIdx, cgIdxRev, ethoTrack, hasPointsTime

In [ ]:
# Clean NaN
G2 = np.array(heatData['G2'])
R2 = np.array(heatData['R2'])
print(G2.shape)

badcols = np.array([x|y for (x,y) in zip(np.isnan(G2).any(axis=0), np.isnan(R2).any(axis=0))])
intG2 = G2[:,~badcols]
intR2 = R2[:,~badcols]

(neurons, time) = intG2.shape
timePoints = np.array(range(1,time))
print(intG2.shape)

f ,ax = plt.subplots(1,2, figsize=(20,5*2))
ax[0].imshow(intG2, aspect=10,cmap="Paired")
ax[1].imshow(intR2, aspect=10,cmap="Paired")
for axis, title in zip(ax, [r'GFP', r'RFP']):
    axis.set_title(title)


# Select some neurons to look at more closely

In [ ]:
# select neurons
plot_neurons = [10, 20]#random_integers(0,neurons, 3)

In [ ]:
# Plot some neuron recordings from GFP and RFP on top of each other
f, ax = plt.subplots(len(plot_neurons),1, figsize=(20,10*len(plot_neurons)))

for i, v in enumerate(plot_neurons):
    ax[i].plot(intG2[v,:], label=("GFP " + str(v)), color='g')
    ax[i].plot(intR2[v,:], label=("RFP " + str(v)), color='r')
    ax[i].legend()
    ax[i].set_title(("Neuron: " + str(v)))

## Some thoughts on plots of GFP on RFP:  

* Doesn't look great. Neurons 0 and 20 match closely, in neuron 10, GFP is often much higher than RFP. Some areas where RFP is also higher. 
* Neuron 50, much higher RFP


In [ ]:
# Looks like RFP is greater than GFP often, let's look at the median
# Don't think this is the best way to do it, but not sure what a better way would be
print(np.median(intG2))
print(np.median(intR2))

f = plt.figure(figsize=(20,10))
ax = plt.axes()
ax.set_title("Median of all neuron values")
ax.plot(np.median(intG2,axis=0), label="GFP", color='g')
ax.plot(np.median(intR2,axis=0), label="RFP", color='r')
ax.legend()

# Not always RFP greater than GFP

# Slopes
Now that we have the data read in, let's look at some basic linear regressions.

In [ ]:
from scipy import stats
import string
X = np.ones((time,1))
Y = np.zeros((time,1))

vals = plot_neurons#[0,56]
f ,ax = plt.subplots(len(vals),1, figsize=(20,len(vals)*10))
for i, v in enumerate(vals):
    print(v)
    Y = intG2[v,:].T
    X = intR2[v,:].T
    slope, intercept, r_value, p_value, std_err = stats.linregress(X,Y)
    ax[i].plot(X,Y, lw=0.2)
    ax[i].scatter(X,Y, c= np.arange(len(X)), cmap='Blues') # other possible colormaps are: YlOrBr, YlGnBu, BuGn
    ax[i].plot(X, intercept + slope*X)
    ax[i].set_title('Neuron: ' + str(v)+', Slope: '+ "%.3f" % slope)

In [ ]:
slopes = np.zeros(neurons)
r_values = np.zeros(neurons)
intercepts = np.zeros(neurons)
resids = np.zeros((neurons,time))

Y = np.empty_like(intG2[1,:].T)
X = np.empty_like(intR2[1,:].T)

for i in range(0,neurons):
    Y = intG2[i,:].T
    X = intR2[i,:].T
    slope, intercept, r_value, p_value, std_err = stats.linregress(X,Y)
    slopes[i] = slope
    r_values[i] = r_value
    intercepts[i] = intercept
    
    resids[i,:] = Y-intercept-slope*X

plt.figure(figsize=(20,10))
plt.scatter(range(0,neurons), slopes)
plt.title('Slopes of G2 regressed on R2 for each neuron')

# find that odd negative slope
# import operator
# min_index, min_value = min(enumerate(slopes), key=operator.itemgetter(1))
# print(min_index)

In [ ]:
# rvalues
plt.figure(figsize=(12,8))

plt.scatter(slopes, r_values)
plt.title('R^2 values of G2 regressed on R2 for each neuron')
plt.xlabel('Slope')
plt.ylabel('r_Value')
plt.ylim([-1,1])


## Take a look at the smallest R value neuron

In [ ]:
import operator
min_index, min_value = min(enumerate(r_values), key=operator.itemgetter(1))

f = plt.figure(figsize=(20,10))
ax = plt.axes()

X = intG2[min_index,:]
Y = intR2[min_index,:]
ax.plot(X, Y, lw=0.2)
ax.scatter(X,Y, c= np.arange(len(X)), cmap='Blues') # other possible colormaps are: YlOrBr, YlGnBu, BuGn
ax.plot(X, intercepts[min_index] + slopes[min_index]*X)
ax.set_title('Neuron: ' + str(min_index)+', Slope: '+ "%.3f" % slopes[min_index])

In [ ]:
# Plot the GFP and RFP recordings
f = plt.figure(figsize=(20,10))
ax = plt.axes()

ax.plot(intG2[min_index,:], label="GFP", color='g')
ax.plot(intR2[min_index,:], label="RFP", color='r')
ax.legend()
ax.set_title(("Neuron: " + str(min_index)))

### Robust estimators

Let's try some robust estimators for the smallest value

In [ ]:
# trying a robust estimator: http://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html#example-linear-model-plot-ransac-py
from sklearn import linear_model

Y = intG2[min_index,:].T
X = intR2[min_index,:].T
# Reshape
Y = np.reshape(Y,(len(Y),1))
X = np.reshape(X,(len(X),1))
model = linear_model.LinearRegression()
model.fit(X,Y)

# Robustly fit linear model with RANSAC algorithm
model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
model_ransac.fit(X, Y)
inlier_mask = model_ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Predict data of estimated models
line_y = model.predict(X)
line_y_ransac = model_ransac.predict(X)

# Compare estimated coefficients
print("Estimated coefficients (normal, RANSAC):")
print(model.coef_, model_ransac.estimator_.coef_)


f = plt.figure(figsize=(20,10))
ax = plt.axes()
ax.plot(X[inlier_mask], Y[inlier_mask], '.', label='Inliers')
ax.plot(X[outlier_mask], Y[outlier_mask], '.', label='Outliers')
ax.plot(X, line_y, label='Linear regressor')
ax.plot(X, line_y_ransac, label='RANSAC regressor')
ax.legend(loc='upper right')

# Redo slope with RANSAC

In [ ]:
ransac_slopes = np.zeros(neurons)
ransac_r_values = np.zeros(neurons)
ransac_intercepts = np.zeros(neurons)

model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())

Y = np.empty((len(intG2[1,:]),1))
X = np.empty((len(intR2[1,:]),1))

for i in range(0,neurons):
    Y = np.reshape(intG2[i,:], (len(intG2[i,:]),1))
    X = np.reshape(intR2[i,:], (len(intG2[i,:]),1))
    
    model_ransac.fit(X, Y)
    line_y = model.predict(X)

    ransac_slopes[i] = model_ransac.estimator_.coef_
    ransac_r_values[i] = model_ransac.score(X,Y)
    ransac_intercepts[i] = model_ransac.estimator_.intercept_

plt.figure(figsize=(20,10))
plt.scatter(range(0,neurons), ransac_slopes)
plt.title('RANSAC: Slopes of G2 regressed on R2 for each neuron')


In [ ]:
# rvalues
plt.figure(figsize=(12,8))

plt.scatter(ransac_slopes, ransac_r_values)
plt.title('RANSAC: R^2 values of G2 regressed on R2 for each neuron')
plt.xlabel('Slope')
plt.ylabel('r_Value')
plt.ylim([-1,1])
